In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re
import html as html_lib

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/6.vehicle.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_request = crawler_series['Url']

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        dict_src, list_src = {}, html.xpath('//img[@itemprop="image"]/@src')
        for i, src in enumerate(list_src):
            dict_src[str(i)] = f'https:{src.strip()}'

        # = = = = = = = = = = = = = = =

        title = html.xpath('//span[@itemprop="name"]/text()')[0].strip()

        # = = = = = = = = = = = = = = =

        div = etree.tostring(html.xpath('//div[@itemprop="offers"]')[0]).decode('utf-8')
        list_info = re.split(r'<br/>|</table>', div)
        list_info = [re.sub(r'<.*?>', '', info) for info in list_info]
        list_info = [re.sub(r' +', ' ', info.strip()) for info in list_info]
        list_info = [html_lib.unescape(info) for info in list_info]

        # = = = = = = = = = = = = = = =

        dict_info, i = {}, 0
        for info in list_info:
            if info.strip() != '':
                dict_info[str(i)] = {info.split(':')[0].strip(): ':'.join(info.split(':')[1:]).strip()}
                i += 1

        # = = = = = = = = = = = = = = =

        dict_price, list_price = {}, html.xpath('//select[contains(@name, "SELECT___")]/option/text()')
        for i, price in enumerate(list_price):
            dict_price[str(i)] = price.strip()

        # = = = = = = = = = = = = = = =

        span = etree.tostring(html.xpath('//span[@itemprop="description"]')[0]).decode('utf-8')
        list_description = span.split('<br/>')
        list_description = [re.sub(r'<.*?>', '', description) for description in list_description]
        list_description = [re.sub(r' +', ' ', description.strip()) for description in list_description]
        list_description = [html_lib.unescape(description) for description in list_description]

        # = = = = = = = = = = = = = = =

        str_description, dict_description, i = '', {}, 0
        for description in list_description:
            if description.strip() != '':
                if ':' in description and description.strip().split(':')[1] != '':
                    dict_description[str(i)] = {description.split(':')[0].strip(): ':'.join(description.split(':')[1:]).strip()}
                    i += 1
                else:
                    str_description += re.sub(r'\n+', '\n', description).strip() + '\n'

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.append({'No': '',
                                      'Title': title,
                                      'Vehicle': crawler_series['Vehicle'],
                                      'Picture': '',
                                      'Url': crawler_series['Url'],
                                      'Json_Src': json.dumps(dict_src),
                                      'Json_Price': json.dumps(dict_price),
                                      'Json_Info': json.dumps(dict_info),
                                      'Description': str_description.strip(),
                                      'Json_Description': json.dumps(dict_description)})

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {url_request}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Title'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct['No'] = range(1, len(df_correct) + 1)
    df_correct.to_excel(f'''./file/7.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Url'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/part_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[True] - [请求1次] - [剩余299条] - [2024/11/25 14:19:19] - https://www.tpms.com/Porsche-911-Turbo-997-2-TSS1-TPMS-p/tss0811-porsche-911turbo997.htm

[True] - [请求1次] - [剩余298条] - [2024/11/25 14:19:20] - https://www.tpms.com/Schrader-33700-EZ-Sensor-TPMS-p/ezc-0206-honda.htm

[True] - [请求1次] - [剩余297条] - [2024/11/25 14:19:20] - https://www.tpms.com/Porsche-Cayenne-Huf-Intellisens-TPMS-p/uvc0811-porsche-cayenne.htm

[True] - [请求1次] - [剩余296条] - [2024/11/25 14:19:20] - https://www.tpms.com/Porsche-Cayman-Huf-Intellisens-TPMS-p/uvc0811-porsche-cayman.htm

[True] - [请求1次] - [剩余295条] - [2024/11/25 14:19:20] - https://www.tpms.com/Porsche-OEM-Huf-TPMS-Sensor-99760602120-433-mhz-p/h0048-porsche.htm

[True] - [请求2次] - [剩余294条] - [2024/11/25 14:19:20] - https://www.tpms.com/Mini-Cooper-OEM-Huf-TPMS-Sensor-36236798726-433MHz-p/h0048-mini.htm

[True] - [请求1次] - [剩余293条] - [2024/11/25 14:19:20] - https://www.tpms.com/Rolls-Royce-Schrader-33700-EZ-Sensor-TPMS-Sensor-p/ezc8238-rollsroyce.htm

[True] - [请求1次